In [1]:
from matplotlib.path import Path
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import random
import time
import json
import cplex._internal._procedural
import cplex
import pandas as pd
from datetime import datetime  
from datetime import timedelta 
import math
from mpl_toolkits import mplot3d
import pickle

import plotly.plotly as py
from heapq import heappush, heappop
from itertools import count

import networkx as nx
import os

In [2]:
path = '/Users/mj/Desktop/INFORMS_CODE_FILE/network design code/Files_to_solve' ## Please set up a directory for the code file.

vdc_index_dic = pickle.load(open(path +'/vdc_index_all_dic.dump', 'rb'))
plant_index_dic = pickle.load(open(path + '/plant_index_dic.dump','rb'))

index_vdc_dic = pickle.load(open(path +'/index_vdc_dic.dump', 'rb'))
index_plant_dic = pickle.load(open(path +'/index_plant_dic.dump', 'rb'))

dist_mat_all = pickle.load(open(path +'/dist_mat_9361.dump','rb'))
location_index_all_dic = pickle.load(open(path +'/location_index_9361.dump', 'rb'))
existing_vdc_dic = pickle.load(open(path +'/Existing_VDC_index_dic.dump', 'rb'))

dealer_index_set_dic = pickle.load(open(path +'/dealer_index_set_dic.dump','rb'))
index_dealer_set_dic = pickle.load(open(path +'/index_dealer_set_dic.dump','rb'))

In [3]:
def assign_dealer_make_d(year,df):

    df["Plant_Arrival_Time"] = pd.to_datetime(df['Plant_Arrival_Time'],infer_datetime_format=True)
    df_new = df[(df['Plant_Arrival_Time'] >= f'{year}-01-01') & (df['Plant_Arrival_Time'] < f'{year+1}-01-01')]

    dealer_d_dic = {}
    for i in zip(df_new["Plant"],df_new["Dealer"]):
        dealer_d_dic[(i[0],i[1])] = []
    for i in zip(df_new["Plant"],df_new["Dealer"]):
        dealer_d_dic[(i[0],i[1])].append(1)
    for i in dealer_d_dic:
        dealer_d_dic[i] = len(dealer_d_dic[i])
        
    dealer_index_dic = {}
    index_dealer_dic = {}

    index = 0
    for i in df_new["Dealer"]:
        dealer_index_dic[i] = 1

    for i in dealer_index_dic:
    #     print(i)
        dealer_index_dic[i] = index
        index_dealer_dic[index] = i
        index += 1
        
    return dealer_d_dic, df_new, dealer_index_dic, index_dealer_dic


def make_dk(dealer_d_dic,index_plant_dic,dealer_index_set_dic,index_vdc_dic):
    
    P = [plant for plant in range(len(index_plant_dic))]

    VDC = [vdc for vdc in range(len(index_vdc_dic))]

    D = [dealer for dealer in range(len(dealer_index_set_dic))]
    
    plant_dealer_finalvdc_dic = pickle.load(open(path + f'/plant_dealer_finalvdc_dic_{Q}_{assign_alpha}.dump','rb'))

    plant_vdc_dealer_dic = {}

    for i in dealer_d_dic:
        plant_vdc_dealer_dic[i] = plant_dealer_finalvdc_dic[i]
        
    sample_dic = {}
    for i in plant_vdc_dealer_dic:
        try:
            sample_dic[i] = [plant_vdc_dealer_dic[i],dealer_d_dic[i]]
        except:
            a = 0   

    sample_dic2 = {}

    for i in sample_dic:
        sample_dic2[(i[0],sample_dic[i][0])] = []

    for i in sample_dic:
        sample_dic2[(i[0],sample_dic[i][0])].append(sample_dic[i][1])
    for i in sample_dic2:
        sample_dic2[i] = sum(sample_dic2[i])

    Plant = ['3A','FF','RO','SO']
    d_k_plant_plant = {}
    d_k_plant_finalvdc = {}
    index = 0
    for i in sample_dic2:
        if i[0] != i[1]:
            d_k_plant_finalvdc[index,i] = sample_dic2[i]
            index += 1
        else:
            d_k_plant_plant[i[0]] = sample_dic2[i]
            
            
    return d_k_plant_finalvdc,d_k_plant_plant,plant_vdc_dealer_dic


def formulate_ND_MIP(d_k_plant_finalvdc,d_k_plant_plant,vdc_index_dic,index_vdc_dic,dist_mat_all,location_index_all_dic):
    
    openvdc = pickle.load(open(path + f'/openvdc_{Q}_{assign_alpha}.dump','rb'))
 
    vdc_index_dic2 = {}
    index = 0
    for i in openvdc:
        vdc_index_dic2[index_vdc_dic[i]] = [index,vdc_index_dic[index_vdc_dic[i]][1]]
        index +=1
        
    vdc_index_dic = vdc_index_dic2
        
    Plant = ['3A','FF','RO','SO']
    
    P = []
    for p in vdc_index_dic:
        if p in Plant:
            P.append(vdc_index_dic[p][0])
    
    a=0
    b=0
    c=0
    d=0
    for i in d_k_plant_finalvdc:
        if i[1][0] == '3A':
            a += d_k_plant_finalvdc[i]
        if i[1][0] == 'FF':
            b += d_k_plant_finalvdc[i]
        if i[1][0] == 'RO':
            c += d_k_plant_finalvdc[i]
        if i[1][0] == 'SO':
            d += d_k_plant_finalvdc[i]

    p_d = {'3A':a,'FF':b,'RO':c,'SO':d}
    D= {}
    for i in p_d:
        D[vdc_index_dic[i][0]] = [p_d[i]]
    for i in d_k_plant_plant:
        if i in Plant:
            D[vdc_index_dic[i][0]].append(d_k_plant_plant[i])
    for i in D:
        D[i] = sum(D[i])
        
    VDC = []
    for i in range(len(vdc_index_dic)):
        VDC.append(i)

    K = []
    for i in range(len(d_k_plant_finalvdc)):
        K.append(i)

    p = {}
    for i in d_k_plant_finalvdc:
        p[i[0]]=i[1][0]
    for i in p :
        p[i] = vdc_index_dic[p[i]][0]

    fv = {}
    for i in d_k_plant_finalvdc:
        fv[i[0]]=i[1][1]
    for i in fv :
        fv[i] = vdc_index_dic[fv[i]][0]


    Alpha=[0.025062576927841394]
    Beta=[0.25825237265642564,-0.01266752,0.0003686]
    Gamma=[3.7681872251915243,-0.0006701078419082377]
    Delta=[2.1676008581272135,-0.00042853714881113004]

    L = 0
    U = 5000

    d=[]
    for i in d_k_plant_finalvdc:
        d.append(d_k_plant_finalvdc[i])

    Distance = {}
    for i in vdc_index_dic:
        Distance[vdc_index_dic[i][0]] = {}
        for j in vdc_index_dic:
            Distance[vdc_index_dic[i][0]][vdc_index_dic[j][0]] = dist_mat_all[location_index_all_dic[i]][location_index_all_dic[j]]

    rail_false_index = []

    for i in vdc_index_dic:
        if vdc_index_dic[i][1] == False:
            rail_false_index.append(vdc_index_dic[i][0])

    TC = {}
    for i in Distance:
        TC[i] = {}
        for j in Distance:
            if Distance[i][j] == 0:
                TC[i][j] = 0
            else:
                if i not in rail_false_index and j not in rail_false_index:
                    TC[i][j] = ((Distance[i][j]*3)+2000)/20
                else:
                    TC[i][j] = ((Distance[i][j]*4)+200)/10


    T = {}
    for i in Distance:
        T[i] = {}
        for j in Distance:
            if Distance[i][j] == 0:
                T[i][j] = 0
            else:
                if i not in rail_false_index and j not in rail_false_index:
                    T[i][j] = (Distance[i][j]/10/24)
                else:
                    T[i][j] = (Distance[i][j]/30/24)

    AC = 730
    FC = 0
    HC = 50

    M = 10000000
    
    cpx = cplex.Cplex()

    
    cpx.variables.add(obj=[1460]*len(VDC),lb=[0]*len(VDC),names=[f'OF_{i}' for i in VDC])
    cpx.variables.add(obj=[FC]*len(VDC),types=["B"]*len(VDC),names=[f'y_{i}' for i in VDC])
    cpx.variables.add(obj=[AC]*len(VDC),names=[f'c_{i}' for i in VDC])
    cpx.variables.add(obj=[HC]*len(VDC),names=[f'w_{i}' for i in VDC])
    cpx.variables.add(types=["B"]*len(VDC),names=[f'z_{i}' for i in VDC])
    cpx.variables.add(obj=[10]*len(VDC)*len(VDC),names=[f'DT_{i}_{j}' for i in VDC for j in VDC])
    cpx.variables.add(types=["B"]*len(VDC)*len(VDC),names=[f'u_{i}_{j}' for i in VDC for j in VDC])

    cpx.variables.add(obj=[(d[k]*TC[i][j])+(10*d[k]*T[i][j])
                          for j in VDC
                          for i in VDC
                          for k in K],
                      types=["B"]*len(K)*len(VDC)*len(VDC), 
                      names=[f'x_{i}_{j}_{k}' 
                            for j in VDC
                            for i in VDC
                            for k in K])
    
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'OF_{i}']+[f'c_{i}']+[f'w_{i}']+[f'z_{i}'],[1]+[1]+[-Alpha[0]]+[-M]) 
                  for i in VDC],
        senses=['G']*len(VDC) ,
        rhs=[-M]*len(VDC),
        names=[f'OF_1_{i}' for i in VDC])
    
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'c_{i}']+[f'w_{i}']+[f'z_{i}'],[Beta[1]]+[Beta[2]]+[-M]) 
                  for i in VDC],
        senses=['G']*len(VDC) ,
        rhs=[Beta[0]-M]*len(VDC),
        names=[f'OF_2_{i}' for i in VDC])
    
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'c_{i}']+[f'w_{i}']+[f'z_{i}'],[Beta[1]]+[Beta[2]]+[-M]) 
                  for i in VDC],
        senses=['L']*len(VDC) ,
        rhs=[Beta[0]]*len(VDC),
        names=[f'OF_3_{i}' for i in VDC])

    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'DT_{i}_{j}']+[f'u_{i}_{j}']+[f'x_{i}_{j}_{k}' for k in K],
            [1]+[-Gamma[0]]+[-(Gamma[1]*d[k]) for k in K]) 
                  for j in VDC
                  for i in VDC],
        senses=['G']*len(VDC)*len(VDC) ,
        rhs=[0]*len(VDC)*len(VDC),
        names=[f'DT_1_{i}_{j}' for i in VDC for j in VDC])
    
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'DT_{i}_{j}']+[f'u_{i}_{j}']+[f'x_{i}_{j}_{k}' for k in K],
            [1]+[-Delta[0]]+[-(Delta[1]*d[k])for k in K]) 
                  for j in VDC
                  for i in VDC],
        senses=['G']*len(VDC)*len(VDC) ,
        rhs=[0]*len(VDC)*len(VDC),
        names=[f'DT_2_{i}_{j}' for i in VDC for j in VDC])
    
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'u_{i}_{j}']+[f'x_{i}_{j}_{k}' for k in K],
            [-M]+[1]*len(K))
                  for j in VDC
                  for i in VDC],
        senses=['L']*len(VDC)*len(VDC) ,
        rhs=[0]*len(VDC)*len(VDC),
        names=[f'DT_3_{i}_{j}' for i in VDC for j in VDC])

    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'DT_{i}_{j}']+[f'x_{i}_{j}_{k}' for k in K],
            [1]+[-(2+d[k]) for k in K])
                  for j in VDC
                  for i in VDC],
        senses=['G']*len(VDC)*len(VDC) ,
        rhs=[0]*len(VDC)*len(VDC),
        names=[f'DT_4_{i}_{j}' for i in VDC for j in VDC])
    
    
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'w_{i}']+[f'x_{j}_{i}_{k}' for j in VDC for k in K],
            [1]+[-d[k] for k in K]*len(VDC))
                  for i in P],
        senses=['E']*len(P) ,
        rhs=[D[i] for i in P],
        names=[f'w_1_{i}' for i in P])
    
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'w_{i}']+[f'x_{j}_{i}_{k}' for j in VDC for k in K],
            [1]+[-d[k] for k in K]*len(VDC))
                  for i in list(set(VDC)-set(P))],
        senses=['E']*len(list(set(VDC)-set(P))) ,
        rhs=[0]*len(list(set(VDC)-set(P))),
        names=[f'w_2_{i}' for i in list(set(VDC)-set(P))])
                         
                         
    for k in K:        
        thevars = []
        thecoefs = []
        for j in VDC:
            if p[k] != j:
                thevars.append('x_%d_%d_%d' %(p[k],j,k))
                thecoefs.append(1)

        cpx.linear_constraints.add(
            lin_expr=[cplex.SparsePair(thevars, thecoefs)],
            senses='E',
            rhs=[1],
            names=['plant_out_1_%d'%(k)])

    for k in K:        
        thevars = []
        thecoefs = []
        for j in VDC:
            if fv[k] != j:
                thevars.append('x_%d_%d_%d' %(j,fv[k],k))
                thecoefs.append(1)

        cpx.linear_constraints.add(
            lin_expr=[cplex.SparsePair(thevars, thecoefs)],
            senses='E',
            rhs=[1],
            names=['final_vdc_in_1_%d'%(k)])

    for k in K:  
        VDC_except = VDC.copy()
        VDC_except.remove(p[k])
        VDC_except.remove(fv[k])

        for i in VDC_except:
            thevars = []
            thecoefs = []
            for j in VDC:
                if i != j:
                    thevars.append('x_%d_%d_%d' %(i,j,k))
                    thecoefs.append(1)
                    thevars.append('x_%d_%d_%d' %(j,i,k))
                    thecoefs.append(-1)

            cpx.linear_constraints.add(
                lin_expr=[cplex.SparsePair(thevars, thecoefs)],
                senses='E',
                rhs=[0],
                names=['in_out_1_%d_%d'%(k,i)])

    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'y_{i}']+[f'c_{i}'],[L]+[-1]) 
                  for i in VDC],
        senses=['L']*len(VDC) ,
        rhs=[0]*len(VDC),
        names=[f'open_1_{i}' for i in VDC])
                         
                         
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'y_{i}']+[f'c_{i}'],[-U]+[1]) 
                  for i in VDC],
        senses=['L']*len(VDC) ,
        rhs=[0]*len(VDC),
        names=[f'open_2_{i}' for i in VDC])
                         
                         
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair(
            [f'x_{i}_{j}_{k}' for i in VDC for j in VDC],
            [1]*len(VDC)*len(VDC))
                  for k in K],
        senses=['L']*len(K) ,
        rhs=[2]*len(K),
        names=[f'x_ijk_num_limit_1_{i}' for k in K])

    for i in VDC:  
        thevars = ['y_%d'%(i)]
        thecoefs = [0.1]
        for k in K:
            for j in VDC:
                    if i != j :
                        thevars.append('x_%d_%d_%d' %(i,j,k))
                        thecoefs.append(-1)
                        thevars.append('x_%d_%d_%d' %(j,i,k))
                        thecoefs.append(-1)

        cpx.linear_constraints.add(
            lin_expr=[cplex.SparsePair(thevars, thecoefs)],
            senses='L',
            rhs=[0],
            names=['fixed cost_1_%d'%(i)])

    for i in VDC:  
        thevars = ['y_%d'%(i)]
        thecoefs = [-(len(VDC)*len(VDC)*len(K))]
        for k in K:
            for j in VDC:
                    if i != j :
                        thevars.append('x_%d_%d_%d' %(i,j,k))
                        thecoefs.append(1)
                        thevars.append('x_%d_%d_%d' %(j,i,k))
                        thecoefs.append(1)

        cpx.linear_constraints.add(
            lin_expr=[cplex.SparsePair(thevars, thecoefs)],
            senses='L',
            rhs=[0],
            names=['fixed cost_2_%d'%(i)])
        

    return cpx,vdc_index_dic2,K,VDC,p,fv,AC,HC,TC,T,d

def solve_ND_MIP(cpx,cpu=4,timelimit=3600):

    start_time= time.time()
    cpx.parameters.threads.set(cpu)
    cpx.parameters.timelimit.set(timelimit)
    # cpx.write("190115.lp")
    cpx.solve()
    num_bnb_nodes = cplex._internal._procedural.getnodecnt(cpx._env._e, cpx._lp)
    num_gap = cplex._internal._procedural.getmiprelgap(cpx._env._e, cpx._lp)    
    sol = cpx.solution
    print("Solution status = ", sol.get_status(), ":", end=' ')
    
    ND_solve_time = time.time() - start_time
    
    return sol,ND_solve_time

def results(Q,vdc_index_dic2,plant_vdc_dealer_dic,sol,ND_solve_time,dealer_d_dic):
    
    index_vdc_dic2 ={}
    for i in vdc_index_dic2:
        index_vdc_dic2[vdc_index_dic2[i][0]] =i
        
        
    routes = {}
    
    for i in plant_vdc_dealer_dic:
        if i[0] == plant_vdc_dealer_dic[i]:
            routes[i] = [i[0],i[1]]
        else:
            routes[i] = []
            routes[i].append([plant_vdc_dealer_dic[i],i[1]])
            
    for k in K:
        for i in VDC:
            for j in VDC:
                if sol.get_values('x_%d_%d_%d' %(i,j,k)) > 0.9:
                    for pp in plant_vdc_dealer_dic:
                        if pp[0] == index_vdc_dic2[p[k]] and plant_vdc_dealer_dic[pp] == index_vdc_dic2[fv[k]] and [index_vdc_dic2[i],index_vdc_dic2[j]] not in routes[pp]:
                            routes[pp].append([index_vdc_dic2[i],index_vdc_dic2[j]])
                            
    capacity = {}
    
    for i in VDC:
        capacity[index_vdc_dic2[i]] = int(sol.get_values('c_%d' %(i)))
        
    total_AC = []
    total_HC = []
    total_DT = []
    total_TC = []
    total_T = []
    total_LT = []
    total_VDC = []

    
    for i in VDC:
        total_AC.append(AC*sol.get_values('c_%d' %(i)))
        total_HC.append(HC*sol.get_values('w_%d' %(i)))

    for i in VDC:
        for j in VDC:
            total_DT.append(10*(sol.get_values('DT_%d_%d' %(i,j))))

    for k in K:
        for i in VDC:
            for j in VDC:
                if sol.get_values('x_%d_%d_%d' %(i,j,k)) > 0.9:
                    total_TC.append(d[k]*TC[i][j])
                    total_T.append(10*d[k]*T[i][j])

    total_LT = sum(total_DT) + sum(total_T)
    total_VDC = sum(total_AC) + sum(total_HC)
    total_TC = sum(total_TC)

    if Q >=44:
        Q_num = Q-44
    else:
        Q_num = 0

    milkrun = []
    milkrun2 = []

    for i in plant_vdc_dealer_dic:
        milkrun.append((((dist_mat_all[location_index_all_dic[str(i[1])]][location_index_all_dic[plant_vdc_dealer_dic[i]]]*4)+200)/10)*dealer_d_dic[i])
        milkrun2.append((((dist_mat_all[location_index_all_dic[str(i[1])]][location_index_all_dic[plant_vdc_dealer_dic[i]]]*4)+200)/10))

    milkrun_cost = sum(milkrun)
    milkrun_cost2 = sum(milkrun2)

    return {

       'ND_solve_time' :  ND_solve_time,
       'Total_logistic_cost' : total_TC,
       'Total_Leadtime_cost' : total_LT,
       'Total_VDC_cost' : total_VDC+((Q_num)*1000000),
       'obj': sol.get_objective_value(),
       'capacity' : capacity,
       'routes' : routes,
       'Total_Milkrun_cost' : milkrun_cost,
       'dealer_d_dic' : dealer_d_dic,
       'plant_vdc_dealer_dic' : plant_vdc_dealer_dic,
       'Milkrun_cost' : milkrun_cost2
    
    }


def dump_files(testtest,dist_mat_all,location_index_all_dic):

    test_dic = {}
    
    for i in testtest["routes"]:

        test_dic[i] = {}
        test_dic[i]["path"] =[]
        test_dic[i]["time"] =[]
        test_dic[i]["path"].append([])

        for j in testtest["routes"][i]:

            try:
                test_dic[i]["path"][0].append((j[0],j[1]))
                if j[0] in ['3F','RX','QW'] or j[1] in ['3F','RX','QW']:
                    test_dic[i]["time"].append((2+(dist_mat_all[location_index_all_dic[str(j[0])]][location_index_all_dic[str(j[1])]]/30/24)))
                else:
                    test_dic[i]["time"].append((2+(dist_mat_all[location_index_all_dic[str(j[0])]][location_index_all_dic[str(j[1])]]/10/24)))

            except:
                test_dic[i]["path"][0]= [i]

                if i[0] in ['3F','RX','QW'] or i[1] in ['3F','RX','QW']:
                    test_dic[i]["time"].append((2+(dist_mat_all[location_index_all_dic[str(i[0])]][location_index_all_dic[str(i[1])]]/30/24)))
                    break
                else:
                    test_dic[i]["time"].append((2+(dist_mat_all[location_index_all_dic[str(i[0])]][location_index_all_dic[str(i[1])]]/10/24)))
                    break

        test_dic[i]["time"] = [sum(test_dic[i]["time"])-2]
    
            
    test_capa = {}

    for i in testtest["capacity"]:
        test_capa[i] = {}
        test_capa[i]["capacity"] = testtest["capacity"][i]
        if i not in ['3F','RX','QW']:
            test_capa[i]["rail"] = True
        else:
            test_capa[i]["rail"] = False

    return test_dic,test_capa



In [4]:
Qs = [44]

assign_alphas = [0.65]

years = [2012]
        
for Q in Qs:
    for assign_alpha in assign_alphas:
        for year in years:
            
            if year == 2011 or year == 2012:
                df = pd.read_csv(path +'/VehicleShipmentRequirement_DataSet1.csv')
            if year == 2013 or year == 2014:
                df = pd.read_csv(path +'/VehicleShipmentRequirement_DataSet2.csv')
            if year == 2015 or year == 2016:
                df = pd.read_csv(path +'/Problem_VehicleShipmentRequirement.csv')
                
            dealer_d_dic, df_new, dealer_index_dic, index_dealer_dic = assign_dealer_make_d(year,df)
            d_k_plant_finalvdc,d_k_plant_plant,plant_vdc_dealer_dic = make_dk(dealer_d_dic,index_plant_dic,dealer_index_set_dic,index_vdc_dic)
   
            cpx,vdc_index_dic2,K,VDC,p,fv,AC,HC,TC,T,d = formulate_ND_MIP(d_k_plant_finalvdc,d_k_plant_plant,vdc_index_dic,index_vdc_dic,dist_mat_all,location_index_all_dic)
            sol,ND_solve_time = solve_ND_MIP(cpx,cpu=4,timelimit=3600)
            results_all = results(Q,vdc_index_dic2,plant_vdc_dealer_dic,sol,ND_solve_time,dealer_d_dic)

            pickle.dump(results_all, open(path + f'/Q{Q}_alpha{assign_alpha}_{year}.dump', 'wb'))


CPXPARAM_TimeLimit                               3600
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
Tried aggregator 1 time.
MIP Presolve eliminated 659 rows and 281366 columns.
MIP Presolve modified 2021837 coefficients.
Reduced MIP has 14592 rows, 30550 columns, and 194434 nonzeros.
Reduced MIP has 28611 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 3.46 sec. (5800.41 ticks)
Found incumbent of value 4.6086188e+09 after 3.83 sec. (6086.82 ticks)
Probing time = 0.04 sec. (10.75 ticks)
Tried aggregator 1 time.
MIP Presolve modified 1106 coefficients.
Reduced MIP has 14592 rows, 30550 columns, and 194434 nonzeros.
Reduced MIP has 28611 binaries, 41 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.18 sec. (133.68 ticks)
Probing time = 0.03 sec. (10.15 ticks)
Clique table members: 76040.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to